In [70]:
import pandas as pd
import string 
from math import log10
df_reviews = pd.read_csv('IMDB_Dataset.csv')

df_reviews_teste = df_reviews.sample(97)

df_filtrado_positivo = df_reviews_teste[df_reviews_teste['sentiment']=='positive']
df_filtrado_negativo = df_reviews_teste[df_reviews_teste['sentiment']=='negative']


dict_palavras_pos={}
for review in df_filtrado_positivo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=set(review.lower().split())
    for palavra in review:
        if palavra in dict_palavras_pos:
            dict_palavras_pos[palavra]+=1
        else:
            dict_palavras_pos[palavra]=1


dict_palavras_neg={}
for review in df_filtrado_negativo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=set(review.lower().split())
    for palavra in review:
        palavra=palavra.lower()
        if palavra in dict_palavras_neg:
            dict_palavras_neg[palavra]+=1
        else:
            dict_palavras_neg[palavra]=1



In [51]:
#calcular P(palavra|classe)

#positivo
quant_review_pos = len(df_filtrado_positivo)
dict_prob_dado_pos = {}

for palavra, quant in dict_palavras_pos.items():
    prob = quant/quant_review_pos
    dict_prob_dado_pos[palavra] = prob



#negativo
quant_review_neg = len(df_filtrado_negativo)
dict_prob_dado_neg = {}

for palavra, quant in dict_palavras_neg.items():
    prob = quant/quant_review_neg
    dict_prob_dado_neg[palavra] = prob



#calcular P(nao palavra|classe)

#positvo
dict_prob_n_palavra_dado_pos = {}
for palavra, prob in dict_prob_dado_pos.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_pos[palavra] = prob_n_palavra

#negativo
dict_prob_n_palavra_dado_neg = {}
for palavra, prob in dict_prob_dado_neg.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_neg[palavra] = prob_n_palavra

In [52]:
#Calcular P(palavra)
palavras_totais ={}

for palavra, quant in dict_palavras_pos.items():
    palavras_totais[palavra] = quant
for palavra, quant in dict_palavras_neg.items():
    if palavra in palavras_totais:
        palavras_totais[palavra]+=quant
    else:
        palavras_totais[palavra]=quant

prob_palavras_dict={}
for palavra in palavras_totais:
    p_palavra=palavras_totais[palavra]/len(df_reviews_teste)
    prob_palavras_dict[palavra]=p_palavra

In [83]:
#Calcular P(classe|texto)

p_negativo = 0.5
p_positivo = 0.5
p_total = 1

# for texto in df_reviews_teste['review']:
texto=df_reviews_teste['review']
texto=texto.iloc[1]

smooth = 1

#positivo
for palavra in palavras_totais:
    if palavra in texto:
        # p_total *= dict_prob_dado_pos[palavra]/(prob_palavras_dict[palavra])
        p_total  *= (dict_prob_dado_pos.get(palavra, 0) + smooth) / (prob_palavras_dict.get(palavra, 0) + smooth * 2)

    else:
        # p_total *= dict_prob_n_palavra_dado_pos[palavra]/(1-prob_palavras_dict[palavra])
        p_total *= (dict_prob_n_palavra_dado_pos.get(palavra, 0) + smooth) / (1-prob_palavras_dict.get(palavra, 0) + smooth * 2)
        print(p_total)
p_total*=p_positivo

print(p_total,df_reviews_teste['sentiment'].iloc[1])

0.6626056338028168
0.13543287105955745
0.08485369999225659
0.05658681118233611
0.03747607915751543
0.02482561731507607
0.016445457714084538
0.010891429684714056
0.004780521422423894
0.0031660239461749414
0.002076198238505708
0.0008294881311409232
0.0003651700633877712
0.00018870840742929627
0.00011988299601547053
7.967033984336078e-05
5.090681794276966e-05
3.326265625163331e-05
2.188988374665791e-05
8.405009468529705e-06
5.566434891468328e-06
2.4346689378083515e-06
1.5975724706618917e-06
1.0474954269308528e-06
6.937309368839296e-07
4.594412542687156e-07
3.0427685274086037e-07
2.0151521495658496e-07
1.2985753187586925e-07
8.600150886923947e-08
5.6389421390290886e-08
3.734535266282506e-08
1.617215577779273e-08
9.82031150441739e-09
5.94461112195973e-09
3.95171142135309e-09
2.555440052474998e-09
1.6928232152492913e-09
1.1211159542095822e-09
2.9376035753586605e-10
1.0597725404778093e-10
6.6721679491818e-11
2.610045883431551e-11
1.0045233454215763e-11
6.546079159855352e-12
4.335310218764893e